# **LAB 01-Feed Forward & Back-Propagation Learning Algorithm**

### **20th January 2025**
# **REDDIPALLI SAI CHARISH**
# **CS22B1095**


---



---



---







#**QUESTION-1**
Implement the Perceptron algorithm from scratch in Python.
*   Initialize the weights with [0 0 0] and a learning rate of 0.0001.
*   For each iteration, calculate the output of the Perceptron for each input in the training set.
*  Use MSE to computer the error for all samples
* Update the weights using the gradient descent procedure.
*   Repeat the above steps until the Perceptron converges or a maximum number of iterations is reached.
* Test the trained Perceptron on a separate test set, explain how you came up with the test set.
* Use the step function as an  activation function in the output layer
Use the IRIS Dataset for the above, considering all four features: sepal length, sepal width, petal length, and petal width, but only two classes -  Setosa, and Versicolor.  Drop the feature vectors of the other class.


In [161]:
import numpy as np
import pandas as pd

## **Perceptron**

In [162]:
class Perceptron:
  def __init__(self,lr=0.0001, epochs=100):
    self.lr = lr
    self.epochs = epochs
    self.acv_func = self.step_func
    self.w = None

  def step_func(self, x):
    return np.where(x >= 0, 1, 0)

  def fit(self,X,y):
    X=np.insert(X,0,1,axis=1)
    n_samples,n_features = X.shape
    self.w = np.zeros(n_features)

    while self.epochs > 0:
      flag = 0
      for i,x_i in enumerate(X):
        dot_output = np.dot(x_i,self.w)
        y_pred = self.acv_func(dot_output)
        if y_pred != y[i]:
          self.w += self.lr * (y[i] - y_pred) * x_i
          flag = 1
      if flag == 0:
        break
      self.epochs -= 1

  def predict(self,X):
    X=np.insert(X,0,1,axis=1)
    return self.acv_func(np.dot(X,self.w))


In [163]:

iris_df = pd.read_csv("iris.csv").drop("Id", axis=1)
iris_df = iris_df[iris_df['Species'].isin(['Iris-setosa', 'Iris-versicolor'])]
print(f"Count: {len(iris_df)}")
iris_df

Count: 100


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,Iris-versicolor
96,5.7,2.9,4.2,1.3,Iris-versicolor
97,6.2,2.9,4.3,1.3,Iris-versicolor
98,5.1,2.5,3.0,1.1,Iris-versicolor


In [164]:

X = iris_df.drop("Species", axis=1).values
y = iris_df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1}).values

In [165]:

def train_test_split(X, y, test_size=0.9):
    n_classes = len(np.unique(y))
    X_train, X_test, y_train, y_test = [], [], [], []
    for i in range(n_classes):
        X_class = X[y == i]
        y_class = y[y == i]
        n_samples = len(X_class)
        n_train = int(n_samples * (1 - test_size))
        X_train.extend(X_class[:n_train])
        X_test.extend(X_class[n_train:])
        y_train.extend(y_class[:n_train])
        y_test.extend(y_class[n_train:])
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Test dataset for each class:")
print(f"{np.asarray(np.unique(y_test, return_counts=True)).T}")



Test dataset for each class:
[[ 0 10]
 [ 1 10]]


In [166]:
perceptron=Perceptron(lr=0.0001,epochs=100)
perceptron.fit(X_train,y_train)

In [167]:
y_pred=perceptron.predict(X_test)
result=pd.DataFrame({'Actual':y_test,'Predicted':y_pred})
result


,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [168]:
accuracy=np.mean(y_pred==y_test)*100
accuracy

100.0

# **QUESTION 2**
Implement the feedforward and backpropagation learning algorithm for multiple perceptrons in Python for the question provided in the attached image.



*   Initialize the weights and biases randomly.

*  Implement the forward pass.

*   Compute the loss between the predicted output and the actual output using an appropriate loss function.
*  Compute the loss between the predicted output and the actual output using an appropriate loss function.


*   Update the weights and biases.


*   Iterate over multiple times (epochs), performing forward propagation, loss calculation, backpropagation, and parameter updates in each iteration till convergence.



##Initialize the weights and biases randomly.

In [169]:
w_0 = np.array([[0.5, 1.5, 0.8], [0.8, 0.2, -1.6]])
w_1 = np.array([[0.9, -1.7, 1.6], [1.2, 2.1, -0.2]])

##feature vectors

In [170]:
X1 = np.array([1.0, 0.7, 1.2])
X2 = np.array([1.0, 0.0, 0.0])
t = np.array([1.0, 0.0])

##sigmoid function

In [171]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [172]:
def sigmoid_output(y: np.array):
    return np.array([1 if i > 0.5 else 0 for i in y])

##Implement the forward pass.

In [173]:
def feed_forward(feature_vector, weight_vector):
    return sigmoid(np.dot(weight_vector, feature_vector))

In [174]:
def back_propagation(W, X, delta_vector, learning_rate):
    n, m = W.shape
    for j in range(n):
        for i in range(m):
            W[j, i] = W[j, i] - learning_rate * delta_vector[j] * X[i]
    return W

In [175]:
def delta_hidden_layer(next_delta_vector, W, output_vector):
    n, m = W.shape
    delta_vector = np.zeros(m - 1)

    for i in range(m - 1):
        for j in range(n):
            delta_vector[j] += next_delta_vector[i] * W[i , j + 1]

    delta_vector = output_vector * (1 - output_vector) * delta_vector
    return delta_vector

In [176]:
def mlp(X1, X2, w_0, w_1, t, learning_rate = 0.001, max_epochs = 1000):
     for epoch in range(max_epochs):
          print(f"Epoch {epoch + 1}")
          o1 = feed_forward(X1, w_0)
          X2[1], X2[2] = o1[0], o1[1]
          o2 = feed_forward(X2, w_1)
          output = sigmoid_output(o2)
          if(np.array_equal(output, t)):
               print(f"Converged after {epoch + 1} epochs")
               break
          else:
               delta_2_vector = (o2 - t) * o2 * (1 - o2)
               # print(f"Delta 2: {delta_2_vector}")
               w_1 = back_propagation(w_1, X2, delta_2_vector, learning_rate)
               # print("O1 : ", o1)
               delta_1_vector = delta_hidden_layer(delta_2_vector, w_1, o1)
               # print(f"Delta 1: {delta_1_vector}")
               w_0 = back_propagation(w_0, X1, delta_1_vector, learning_rate)
          print(w_0, '\n\n' , w_1)
     else:
          print("Did not converge after max epochs")
     return w_0, w_1

In [177]:
w_0, w_1 = mlp(X1, X2, w_0, w_1, t, 0.0001, 10)
print("")
print("")

print("Final weight vector:")
print("w_0:", w_0)
print("w_1:", w_1)

Epoch 1
[[ 0.49999779  1.49999845  0.79999735]
 [ 0.80000453  0.20000317 -1.59999456]] 

 [[ 0.90001377 -1.69998726  1.60000376]
 [ 1.19999601  2.09999631 -0.20000109]]
Epoch 2
[[ 0.49999558  1.49999691  0.7999947 ]
 [ 0.80000906  0.20000634 -1.59998913]] 

 [[ 0.90002755 -1.69997452  1.60000752]
 [ 1.19999202  2.09999262 -0.20000218]]
Epoch 3
[[ 0.49999337  1.49999536  0.79999204]
 [ 0.80001359  0.20000952 -1.59998369]] 

 [[ 0.90004132 -1.69996178  1.60001128]
 [ 1.19998803  2.09998893 -0.20000327]]
Epoch 4
[[ 0.49999116  1.49999381  0.79998939]
 [ 0.80001812  0.20001269 -1.59997825]] 

 [[ 0.90005509 -1.69994905  1.60001503]
 [ 1.19998404  2.09998524 -0.20000436]]
Epoch 5
[[ 0.49998895  1.49999226  0.79998674]
 [ 0.80002266  0.20001586 -1.59997281]] 

 [[ 0.90006887 -1.69993631  1.60001879]
 [ 1.19998005  2.09998155 -0.20000544]]
Epoch 6
[[ 0.49998674  1.49999072  0.79998409]
 [ 0.80002719  0.20001903 -1.59996738]] 

 [[ 0.90008264 -1.69992357  1.60002255]
 [ 1.19997606  2.09997786 